<h1><center>Interactive Map - Denver Crime (2019)</center></h1>

## Motivation
In this tutorial the goal is to both learn how to create interactive maps with the folium package. As part of this exercise I also hope to visualize Denver crime data.

First we'll import the necessary libraries and global variables.

In [1]:
import folium
from folium import Choropleth, Circle, Marker
from folium.plugins import HeatMap, MarkerCluster
import pandas as pd
from datetime import datetime
import math
import geopandas as gpd

denver_coordinates = 39.742043,-104.991531

## Getting Started
We begin by creating a relatively simple map of Denver with folium.Map().

In [2]:
# Create a map of Denver
map_denver = folium.Map(location=denver_coordinates, tiles='openstreetmap', zoom_start=12)

# Display map of Denver
map_denver

Several arguments customize the appearance of the map:
<ul>
<li>location sets the initial center of the map. We use the latitude (39.742043° N) and longitude (-104.991531° E) of the city of Denver.</li>
<li>tiles changes the styling of the map; in this case, we choose the OpenStreetMap style. If you're curious, you can find the other options listed here.</li>
<li>zoom_start sets the initial level of zoom of the map, where higher values zoom in closer to the map.</li>
</ul

    Take the time now to explore by zooming in and out, or by dragging the map in different directions.

## The Data
Now, we'll add some crime data to the map!

We won't focus on the data loading step. Instead, you can imagine you are at a point where you already have the data in a pandas DataFrame crimes. The first five rows of the data are shown below.

This dataset was obtained through 




## Cleaning the Data

In [3]:
# Load the dataset
denver_crimes = pd.read_csv("DenverCrime2019.csv", encoding='latin-1')

# Drop rows with missing locations
denver_crimes.dropna(subset=['GEO_LAT', 'GEO_LON', 'DISTRICT_ID'], inplace=True)

# Next we slect a handful of the more major types of crimes crimes in 2018
denver_crimes = denver_crimes[denver_crimes.OFFENSE_TYPE_ID.isin([
    'homicide-family', 'homicide-other', 'kidnap-juvenile-victim', 'kidnap-adult-victim', 'kidnap-dv', 'sex-aslt-rape', 'sex-aslt-statutory-rape', 'sex-aslt-non-rape', 
    'sex-asslt-sodomy-man-strng-arm', 'sex-aslt-statutory-rape', 'sex-aslt-w-object', 'robbery-business', 'robbery-street', 'robbery-residence', 'robbery-bank', 'robbery-car-jacking', 
    'assault-dv', 'weapon-fire-into-occ-bldg', 'aggravated-assault', 'aggravated-assault-dv', 'menacing-felony-w-weap', 'arson-residence', 'burglary-residence-by-force', 
    'burglary-business-by-force', 'sex-aslt-rape'])]
#denver_crimes = denver_crimes[denver_crimes.YEAR>=2019]

# Print the first five rows of the table
denver_crimes.head()

,incident_id,offense_id,OFFENSE_CODE,OFFENSE_CODE_EXTENSION,OFFENSE_TYPE_ID,OFFENSE_CATEGORY_ID,FIRST_OCCURRENCE_DATE,LAST_OCCURRENCE_DATE,REPORTED_DATE,INCIDENT_ADDRESS,GEO_X,GEO_Y,GEO_LON,GEO_LAT,DISTRICT_ID,PRECINCT_ID,NEIGHBORHOOD_ID,IS_CRIME,IS_TRAFFIC,VICTIM_COUNT
22594,2.019415e+09,2.020000e+15,902,0,homicide-family,murder,7/3/2019 0:01,7/3/2019 12:34,7/3/2019 13:46,4755 N PEARL ST,3146039.0,1710465.0,-104.980378,39.782887,1,112,globeville,1,0,1
22595,2.019257e+09,2.020000e+15,902,0,homicide-family,murder,4/25/2019 3:22,NaN,4/25/2019 3:22,2718 W 28TH AVE,3134681.0,1700950.0,-105.020969,39.756938,1,121,jefferson-park,1,0,1
22596,2.019577e+09,2.020000e+15,902,0,homicide-family,murder,9/9/2019 16:46,NaN,9/9/2019 18:58,1255 19TH ST,3142577.0,1699294.0,-104.992918,39.752275,6,612,union-station,1,0,1
22597,2.019305e+09,2.020000e+15,902,0,homicide-family,murder,5/15/2019 21:34,NaN,5/15/2019 21:34,3015 N ADAMS ST,3154873.0,1701958.0,-104.949130,39.759391,2,213,skyland,1,0,1
22598,2.019454e+09,2.020000e+15,902,0,homicide-family,murder,7/19/2019 20:35,NaN,7/19/2019 23:48,3212 N DEXTER ST,3159422.0,1703065.0,-104.932926,39.762353,2,221,northeast-park-hill,1,0,1


## Plotting Crime Points
In the interest of saving space, we'll mark just the burgleries of businesses by force (burglary-business-by-force).  

First we'll create a holder for those sepecific crimes.

In [4]:
date_format = "%m/%d/%Y %H:%M"


bbbf_crimes = denver_crimes[((denver_crimes.OFFENSE_TYPE_ID == 'burglary-business-by-force') & \
                      pd.DatetimeIndex(denver_crimes['REPORTED_DATE']).hour.isin(range(7,17)))] 

#### folium.Marker
We add markers to the map with folium.Marker(). Each marker below corresponds to a different burglary in the city of Denver.

In [5]:
# Create a map with folium
map_bbbf_crimes = folium.Map(location=denver_coordinates, tiles='cartodbpositron', zoom_start=12)

# Add points to the map
for idx, row in bbbf_crimes.iterrows():
    Marker([row['GEO_LAT'], row['GEO_LON']]).add_to(map_bbbf_crimes)

# Display the map
map_bbbf_crimes

#### folium.plugins.MarkerCluster
As you can see, there are so many business burgerlies that the map gets a bit cluttered making it hard to discern. To help with this issue we can use folium.plugins.MarkerCluster() can help to declutter the map. This plugin adds each marker to a MarkerCluster object with a number to represent the number of objects in each cluster.

In [6]:
# Create the MarkCluster map
map_bbbf_cluster = folium.Map(location=denver_coordinates, tiles='cartodbpositron', zoom_start=12)

# Add points to the map
mc = MarkerCluster()
for idx, row in bbbf_crimes.iterrows():
    if not math.isnan(row['GEO_LON']) and not math.isnan(row['GEO_LAT']):
        mc.add_child(Marker([row['GEO_LAT'], row['GEO_LON']]))

map_bbbf_cluster.add_child(mc)

# Display the map
map_bbbf_cluster

As we can see, 

## Bubble Map

A bubble map basically adds some circles at some locations on the map. Those circle must have coordinates (longitude and latitude). They usually have values as well, values that are mapped to the circle size.  We can change the size and color of each circle as well as showing the relationship between locations and other variables.

(change text below)

A bubble map uses circles instead of markers. By varying the size and color of each circle, we can also show the relationship between location and two other variables.

We create a bubble map by using folium.Circle() to iteratively add circles. In the code cell below, robberies that occurred in hours 9-12 are plotted in green, whereas robberies from hours 13-17 are plotted in red.

In [7]:
# Create a base map
map_bbbf_base = folium.Map(location=denver_coordinates, tiles='cartodbpositron', zoom_start=13)

#testit = pd.DatetimeIndex(denver_crimes['REPORTED_DATE']).hour
#print(testit)

def color_producer(val):
    if val <= 12:
        return 'forestgreen'
    else:
        return 'darkred'

# Add a bubble map to the base map
for i in range(0,len(bbbf_crimes)):
    Circle(
        location=[bbbf_crimes.iloc[i]['GEO_LAT'], bbbf_crimes.iloc[i]['GEO_LON']]
        radius=20, color=color_producer(pd.DatetimeIndex(bbbf_crimes.iloc[i]['REPORTED_DATE']).hour).add_to(map_bbbf_base)
        
        #radius=20, color=color_producer(pd.DatetimeIndex(bbbf_crimes.iloc[i]['REPORTED_DATE'].hour)).add_to(map_bbbf_base))
        #pd.DatetimeIndex(denver_crimes['REPORTED_DATE']).hour.isin(range(7,17)))] 
        
# Display the map
map_bbbf_base

SyntaxError: invalid syntax (1459862937.py, line 17)

(change)

Note that folium.Circle() takes several arguments:

location is a list containing the center of the circle, in latitude and longitude.
radius sets the radius of the circle.
Note that in a traditional bubble map, the radius of each circle is allowed to vary. We can implement this by defining a function similar to the color_producer() function that is used to vary the color of each circle.
color sets the color of each circle.
The color_producer() function is used to visualize the effect of the hour on robbery location.

## Heatmaps

(change)

To create a heatmap, we use folium.plugins.HeatMap(). This shows the density of crime in different areas of the city, where red areas have relatively more criminal incidents.

As we'd expect for a big city, most of the crime happens near the center.

In [ ]:
# Create a base map
heat_map_base = folium.Map(location=denver_coordinates, tiles='cartodbpositron', zoom_start=12)

# Add a heatmap to the base map
HeatMap(data=denver_crimes[['GEO_LAT', 'GEO_LON']], radius=10).add_to(heat_map_base)

# Display the map
heat_map_base

As you can see in the code cell above, folium.plugins.HeatMap() takes a couple of arguments:

data is a DataFrame containing the locations that we'd like to plot.
radius controls the smoothness of the heatmap. Higher values make the heatmap look smoother (i.e., with fewer gaps).

## Choropleth maps
To understand how crime varies by police district, we'll create a choropleth map.

As a first step, we create a GeoDataFrame where each district is assigned a different row, and the "geometry" column contains the geographical boundaries.

In [ ]:
# GeoDataFrame with geographical boundaries of Boston police districts
districts_full = gpd.read_file('Police_Districts/Police_Districts.shp')
districts = districts_full[["DIST_NUM", "geometry"]].set_index("DIST_NUM")
districts.head()

In [ ]:
# Number of crimes in each police district
plot_dict = denver_crimes.DISTRICT_ID.value_counts()
plot_dict.head()

(change)


It's very important that plot_dict has the same index as districts - this is how the code knows how to match the geographical boundaries with appropriate colors.

Using the folium.Choropleth() class, we can create a choropleth map. If the map below does not render for you, try viewing the page in a different web browser.

In [ ]:
# Create a base map
district_map = folium.Map(location=denver_coordinates, tiles='cartodbpositron', zoom_start=12)

# Add a choropleth map to the base map
Choropleth(geo_data=districts.__geo_interface__, 
           data=plot_dict, 
           key_on="feature.id", 
           fill_color='YlGnBu', 
           legend_name='Major criminal incidents (Jan-Aug 2018)'
          ).add_to(district_map)

# Display the map
district_map

Note that folium.Choropleth() takes several arguments:

geo_data is a GeoJSON FeatureCollection containing the boundaries of each geographical area.
In the code above, we convert the districts GeoDataFrame to a GeoJSON FeatureCollection with the __geo_interface__ attribute.
data is a Pandas Series containing the values that will be used to color-code each geographical area.
key_on will always be set to feature.id.
This refers to the fact that the GeoDataFrame used for geo_data and the Pandas Series provided in data have the same index. To understand the details, we'd have to look more closely at the structure of a GeoJSON Feature Collection (where the value corresponding to the "features" key is a list, wherein each entry is a dictionary containing an "id" key).
fill_color sets the color scale.
legend_name labels the legend in the top right corner of the map.